In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

In [2]:
def read_data(fname):
    try:
        with open(fname, "rb") as file:
            loaded_data = pickle.load(file)
        return loaded_data["buffer_data"], loaded_data["paraset"]
    except (FileNotFoundError, pickle.UnpicklingError) as e:
        print(f"Error loading file {fname}: {e}")
        return None, None
    

In [3]:
test_data, parameters = read_data("../measurement/output/20250207_odmr_tracking_snr/data_dev_pulsed_odmr_tracking_changedetection_1739314872.4265556_runlen_900.pkl")
idx_pointer = parameters['idx_pointer']
reps=parameters['reps']
t_atrack=parameters['t_ribloc'] + parameters['t_fevol']

In [4]:
from scipy.signal import find_peaks, peak_widths


In [5]:
# Load in a test dataset 
end = 518

# Process the 'test_data' buffer and background arrays
buffer_mean = np.zeros_like(test_data[3])
bg_mean = np.zeros_like(test_data[0])

# For 'buffer' array
buffer_mean[:idx_pointer, :] = test_data[3][:idx_pointer, :] / (reps + 1)
buffer_mean[idx_pointer:, :] = test_data[3][idx_pointer:, :] / reps
data_mean = np.mean(buffer_mean[:, 0:end], axis=1)

# For 'bg' array
bg_mean[:idx_pointer, :] = test_data[0][:idx_pointer, :] / (reps + 1)
bg_mean[idx_pointer:, :] = test_data[0][idx_pointer:, :] / reps
bg_mean = np.mean(bg_mean[:, 0:end], axis=1)


freq = np.fft.fftfreq(len(data_mean), (t_atrack / 1E9))
sig_fft = np.abs(np.fft.fft(np.abs(data_mean)))
sig_fft -= np.abs(np.fft.fft(bg_mean))

positive_freqs = freq > 0
x=freq[positive_freqs]
y=(sig_fft)[positive_freqs]

In [6]:
## Peak finder

peaks, _ = find_peaks(y,height=0.5*np.max(y))
widths, width_heights, left_ips, right_ips = peak_widths(y, peaks, rel_height=0.5)
LR_width = np.ceil(widths/2).astype(int)
idx_peak=peaks[0]

In [7]:
## Noise floor

signal=y[idx_peak]
indicies = np.where(x!= idx_peak)[0]  # Get indices where x[3] is between 20 and 50
lower=np.min(indicies)
higher= np.max(indicies)

signal=np.max(y[lower:higher])

loc = np.where((x < idx_peak - LR_width) | (x > idx_peak + LR_width))[0]
noise = np.mean(np.abs(y[loc]))

print(signal)
print(noise)
print(signal/noise)

0.06271913622786432
0.00357212695598965
17.557924732405855


In [8]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(0, len(y), 1), y=y))

# Add labels
fig.update_layout(
    title="Your Plot Title",  # Optional: Add a title
    xaxis_title="Index",  # Add label for the X-axis
    yaxis_title="Amplitude",  # Add label for the Y-axis
)

fig.show()


In [9]:
## optimize the readoutwindow to get the best SNR

In [12]:
np.shape(test_data[3])
type(test_data[3])

numpy.ndarray

In [ ]:
def SNR_opt(end):

    end = int(end * 1000)


    # Process the 'test_data' buffer and background arrays
    buffer_mean = np.zeros_like(test_data[3])
    bg_mean = np.zeros_like(test_data[0])

    # For 'buffer' array
    buffer_mean[:idx_pointer, :] = test_data[3][:idx_pointer, :] / (reps + 1)
    buffer_mean[idx_pointer:, :] = test_data[3][idx_pointer:, :] / reps
    data_mean = np.mean(buffer_mean[:, 0:end], axis=1)

    # For 'bg' array
    bg_mean[:idx_pointer, :] = test_data[0][:idx_pointer, :] / (reps + 1)
    bg_mean[idx_pointer:, :] = test_data[0][idx_pointer:, :] / reps
    bg_mean = np.mean(bg_mean[:, 0:end], axis=1)


    freq = np.fft.fftfreq(len(data_mean), (t_atrack / 1E9))
    sig_fft = np.abs(np.fft.fft(np.abs(data_mean)))
    sig_fft -= np.abs(np.fft.fft(bg_mean))

    positive_freqs = freq > 0
    x=freq[positive_freqs]
    y=(sig_fft)[positive_freqs]
    
    idx_peak=find_peaks(y,height=0.5*np.max(y))[0][0]
    peaks, _ = find_peaks(y,height=0.5*np.max(y))
    widths, width_heights, left_ips, right_ips = peak_widths(y, peaks, rel_height=0.5)
    LR_width = np.ceil(widths/2).astype(int)
    signal=y[idx_peak]
    indicies = np.where(x!= idx_peak)[0]  # Get indices where x[3] is between 20 and 50
    lower=np.min(indicies)
    higher= np.max(indicies)

    signal=np.max(y[lower:higher])

    loc = np.where((x < idx_peak - LR_width) | (x > idx_peak + LR_width))[0]
    noise = np.mean(np.abs(y[loc]))

    print(signal/noise)
    print(end)
    return -signal/noise
    
    
bounds = [(20 / 1000, 600 / 1000)]
result = differential_evolution(SNR_opt, bounds=bounds,x0=300/1000)

# Print results
if result.success:
    print("Optimization successful!")
    print("Optimal value of x:", result.x)
    print("Objective function value at minimum:", result.fun)
else:
    print("Optimization failed! Reason:", result.message)


C:\Users\HFNV\AppData\Local\Temp\ipykernel_13456\4075563623.py:3: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



15.76053711796132
300


C:\Users\HFNV\AppData\Local\Temp\ipykernel_13456\4075563623.py:30: PeakPropertyWarning:

some peaks have a prominence of 0

C:\Users\HFNV\AppData\Local\Temp\ipykernel_13456\4075563623.py:30: PeakPropertyWarning:

some peaks have a width of 0



4.709323708578116
128
16.157892583891545
357
15.466596702450783
321
8.762188770437188
188
4.388001128348135
53
5.346304159844444
169
17.465004413865593
485
4.404658878879299
87
17.483094906424423
416
17.42903113297299
477
16.42940584620044
586
16.677725691909572
396
14.183852251870613
273
16.962276064603106
543
17.47700700304375
507
16.616079573853156
576
17.283516875872056
434
5.329578370485877
135
17.111795485755785
557
16.955607565225947
538
16.46505823043832
379
17.47700700304375
507
17.114754509330183
555
4.814978013793482
25
8.087996246168652
184
17.193134181319785
461
15.909336351810158
312
17.460624503937193
481
17.07926544342129
450
17.075427921684494
451
17.30536260011428
432
14.94084578434013
342
17.157830248569088
444
17.263263287059264
436
16.157892583891545
357
17.38093765634441
426
14.955025867012017
330
17.263263287059264
436
17.267841984256165
407
16.789758863211397
398
17.231084404439624
439
17.465036948272154
420
17.31540791167153
431
17.483094906424423
416
17.483094